In [2]:
# Step 1: Initialize environment

from dotenv import load_dotenv

load_dotenv()

import numpy as np
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


Weather Model

Taken direclty from the TensorFlow documentation (https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel).

We will model a simple weather system and try to predict the temperature on each day given the following information.

- Cold days are encoded by a 0 and hot days are encoded by a 1.
- The first day in our sequence has an 80% chance of being cold.
- A cold day has a 30% chance of being followed by a hot day.
- A hot day has a 20% chance of being followed by a cold day.
- On each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day.

If you're unfamiliar with standard deviation it can be put simply as the range of expected values.

In this example, on a hot day the average temperature is 15 and ranges from 5 to 25.

To model this in TensorFlow we will do the following.

In [4]:
# Step 2: Setting the distribution parameters using native TensorFlow Hidden Markov Models

# Since TensorFlow Probability has compatibility issues, we'll implement HMM with native TF

# Initial distribution: 80% chance of cold (0), 20% chance of hot (1)
initial_probs = tf.constant([0.8, 0.2])

# Transition matrix: [cold->cold, cold->hot], [hot->cold, hot->hot], former `tfd.Categorical` creation
transition_matrix = tf.constant(
    [
        # Cold day: 70% stay cold, 30% become hot
        [0.7, 0.3],
        # Hot day: 20% become cold, 80% stay hot
        [0.2, 0.8],
    ]
)

# Observation parameters: [cold_mean, hot_mean], [cold_std, hot_std]
obs_means = tf.constant([0.0, 15.0])  # Cold: 0°, Hot: 15°, former `loc` parameter for `tfd.Normal`
obs_stds = tf.constant(
    [5.0, 10.0]
)  # Cold: ±5°, Hot: ±10°, former `scale` parameter for `tfd.Normal`, not used actually

print("Initial probabilities:", initial_probs.numpy())
print("Transition matrix:", transition_matrix.numpy())
print("Observation means:", obs_means.numpy())
print("Observation stds:", obs_stds.numpy())

Initial probabilities: [0.8 0.2]
Transition matrix: [[0.7 0.3]
 [0.2 0.8]]
Observation means: [ 0. 15.]
Observation stds: [ 5. 10.]


We've now created distribution variables to model our system and it's time to create the hidden markov model.



The number of steps represents the number of days that we would like to predict information for. In this case we've chosen 7, an entire week.

To get the expected temperatures on each day we can do the following.


In [6]:
# Step 3: Predict expected temperatures using proper HMM mean calculation


def predict_temperatures_hmm(num_days=7):
    """Calculate HMM mean like original TFP model.mean()"""
    # This matches the original tfd.HiddenMarkovModel behavior
    # where observation_distribution contributes to the overall mean

    state_probs = initial_probs
    expected_temps = []

    for day in range(num_days):
        # Expected observation = sum over states of (state_prob * obs_mean)
        # This is exactly what the original observation_distribution did
        expected_temp = tf.reduce_sum(state_probs * obs_means)
        expected_temps.append(expected_temp.numpy())

        # Forward step: update state probabilities
        state_probs = tf.linalg.matvec(transition_matrix, state_probs, transpose_a=True)

    return expected_temps


# Predict temperatures for a week (equivalent to original model.mean())
temperatures = predict_temperatures_hmm(7)
print("\nPredicted temperatures (HMM mean, equivalent to original TFP):")
for i, temp in enumerate(temperatures, 1):
    print(f"Day {i}: {temp:.2f}°C")


Predicted temperatures (HMM mean, equivalent to original TFP):
Day 1: 3.00°C
Day 2: 6.00°C
Day 3: 7.50°C
Day 4: 8.25°C
Day 5: 8.63°C
Day 6: 8.81°C
Day 7: 8.91°C


Conclusion

So that's it for the core learning algorithms in TensorFlow. Hopefully you've learned about a few interesting tools that are easy to use! To practice I'd encourage you to try out some of these algorithms on different datasets.